In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Road signs Classification

This notebook demonstrates how to classify images from Imagery Insights Road signs dataset using the Gemini 2.5 Flash model via Google Cloud's Vertex AI.

## Install Required Libraries

In [1]:
%pip install --upgrade google-cloud-bigquery google-cloud-aiplatform

## Configuration

**Important**: Replace the placeholder values below with your actual GCP Project ID and Region.

In [ ]:
PROJECT_ID = 'your-gcp-project-id'  # @param {type:"string"}
REGION = 'us-central1'      # @param {type:"string"}

# BigQuery Configuration
BIGQUERY_DATASET_ID = 'your_bq_dataset' # @param {type:"string"}
BIGQUERY_TABLE_ID = 'your_bq_table' # @param {type:"string"}
QUERY_LIMIT = 10 # @param {type:"integer"}
ASSET_TYPE = "ASSET_CLASS_ROAD_SIGN" # @param {type:"string"}
MODEL = "gemini-2.5-flash" # @param {type:"string"}

## Imports and Vertex AI Initialization

In [1]:
import vertexai
from google.cloud import bigquery
from google import genai
from google.genai.types import Content, Part

# Initialize Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=REGION)

## Create vertex client

In [1]:
client = genai.Client(vertexai=True, project=PROJECT_ID, location=REGION)

## Fetch Image URIs from BigQuery

Next, we'll query a BigQuery table to get the GCS URIs of the images we want to classify.

In [1]:
PROMPT = """You will be provided with a photo of a utility pole:
{photo_of_utility_pole}

You are an image analysis expert. Your task is to analyze an image of a road scene and identify any road signs from a predefined list.
InstructionsAnalyze the Image: Carefully analyze the provided image. Your entire analysis MUST be based exclusively on the image provided. Do not infer information or make assumptions beyond what is visually present.
Identify Signs: Scan the image to identify all road signs. For each potential sign, use the following logic:
a. First, identify its shape and color.
b. Next, read any visible text or interpret symbols.
c. Finally, match this combination of features to a category in the 'Sign Categories & Features' list below.
Format the Output: Present your findings as a JSON array. Each object in the array represents one identified sign and must use the schema defined in the 'JSON Output Schema' section.
DefinitionsSign Categories & Features:
Stop: Octagonal (8-sided) shape, red background, white text ('STOP').
Yield: Downward-pointing triangle shape, white background with a red border.
Speed Limit: Vertical rectangle shape, typically white background with black numbers.
Pedestrian Crossing: Pentagon (5-sided) shape or square, typically yellow or fluorescent green/yellow with black symbols of people walking.
No Parking: Typically a rectangle or square, often with a red circle and slash over the letter 'P'.
Turn: Rectangle shape, typically with a black arrow indicating the direction of the turn (left, right, U-turn) on a white or yellow background.
Do not enter: A red circle with a white horizontal bar inside.
Street name: A horizontal rectangle, typically with a green, blue, or white background and white or black text.
Other: Any sign not matching the specific descriptions above.
Confidence Level Definitions:
High: The sign is clearly visible and legible in the image.
Medium: The sign's characteristic shape and/or color are visible, but the text or symbol is unreadable or obscured. The identification is based on shape and color context (e.g., a red octagon is identified as a Stop sign even if the text is unreadable).
JSON Output SchemaYour output must be a valid JSON array where each object represents a single sign and contains the following keys:
\"Sign\": [Name of the sign from the 'Sign Categories & Features' list]
\"Reasoning\": [A brief explanation of the visual evidence (shape, color, text).]
\"Confidence\": [\"High\" or \"Medium\"]
ExampleThis is an illustrative example. Your analysis must be based on the image you are provided.
[
  {
    \"Sign\": \"Stop\",
    \"Reasoning\": \"Identified a red, octagonal sign with the word \\\"STOP\\\" in clear, white text.\",
    \"Confidence\": \"High\"
  },
  {
    \"Sign\": \"Speed Limit\",
    \"Reasoning\": \"Identified a vertical rectangular sign with a white background. The numbers '45' are legible, consistent with a speed limit sign.\",
    \"Confidence\": \"High\"
  },
  {
    \"Sign\": \"Street name\",
    \"Reasoning\": \"Identified a green, horizontal rectangular sign. The text is blurry, but the shape and color are characteristic of a street name sign.\",
    \"Confidence\": \"Medium\"
  }
]
"""

## Define Image Classification Function

This function takes a GCS URI and a prompt, then uses the Gemini 1.5 Flash model to generate a description of the image.

In [1]:
def classify_image_with_gemini(gcs_uri: str, prompt: str) -> str:
    """
    Classifies an image using the Gemini 1.5 Flash model by directly passing its GCS URI.
    """
    try:
          contents = [
          PROMPT,
          Part(file_data={'file_uri': gcs_uri, 'mime_type': 'image/jpeg'})]

          response = client.models.generate_content(model=MODEL, contents=contents)
          return response.text
    except Exception as e:
        print(f"Error classifying image from URI {gcs_uri}: {e}")
        return "Classification failed."

#Get the image URIs from BigQuery

In [1]:
from google.cloud import bigquery

BIGQUERY_SQL_QUERY = f"""
SELECT
  *
FROM
  `{PROJECT_ID}.{BIGQUERY_DATASET_ID}.{BIGQUERY_TABLE_ID}`
  WHERE asset_type = '{ASSET_TYPE}'

LIMIT {QUERY_LIMIT};
"""

# Execute BigQuery Query
try:
    bigquery_client = bigquery.Client(project=PROJECT_ID)
    query_job = bigquery_client.query(BIGQUERY_SQL_QUERY)
    query_response_data = [dict(row) for row in query_job]
    gcs_uris = [item.get("gcs_uri") for item in query_response_data if item.get("gcs_uri")]

    print(f"Successfully fetched {len(gcs_uris)} GCS URIs:")
    for uri in gcs_uris:
        print(uri)
except Exception as e:
    print(f"An error occurred while querying BigQuery: {e}")

## Classify Images

Finally, we loop through the GCS URIs we fetched and pass them to our classification function along with a prompt.

In [1]:
if 'gcs_uris' in locals() and gcs_uris:
    for uri in gcs_uris:
        print(f"--- Classifying {uri} ---")
        classification = classify_image_with_gemini(uri, PROMPT)
        print(f"Result: {classification}\n")
else:
    print("No GCS URIs were found to classify.")